Import & Combine

In [1]:
import pandas as pd
import concurrent.futures

# Define the range of years
start_year = 2012
end_year = 2024

# Function to read a single Excel file
def read_excel_file(year):
    file_path = f"F:/portal_base/data/raw/contratospub{year}.xlsx"
    return pd.read_excel(file_path)

# Use ThreadPoolExecutor to read files in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Map the read_excel_file function to the range of years
    df_list = list(executor.map(read_excel_file, range(start_year, end_year + 1)))

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

# Display the combined DataFrame
combined_df.head()


# Save as pickle file
pkl_output_path = "F:/portal_base/data/processed/combined_data.pkl"
combined_df.to_pickle(pkl_output_path)

Columns wrangling

In [ ]:
import pandas as pd
from tqdm import tqdm

# Load the combined DataFrame from the pickle file
pkl_input_path = "F:/portal_base/data/processed/combined_data.pkl"

combined_df = pd.read_pickle(pkl_input_path)

# Drop columns
combined_df = combined_df.drop(columns=['Observacoes', 'numAcordoQuadro', 'DescrAcordoQuadro', 'concorrentes', 'regime', 'fundamentacao', 'nAnuncio', 'TipoAnuncio', 'idINCM', 'ProcedimentoCentralizado', 'dataFechoContrato', 'linkPecasProc', 'CritMateriais', 'tipoFimContrato', 'justifNReducEscrContrato', 'dataPublicacao'])

# Separar a coluna adjudicante usando a primeira ocorrência de " - "
combined_df['nif_adjudicante'] = combined_df['adjudicante'].str.split(' - ', n=1, expand=True)[0]
combined_df['nome_adjudicante'] = combined_df['adjudicante'].str.split(' - ', n=1, expand=True)[1]

# Remover a coluna original
combined_df.drop(columns=['adjudicante'], inplace=True)

# Separar a coluna cpv em código e descrição
combined_df[['cpv_código', 'cpv_descricao']] = combined_df['cpv'].str.split(' - ', n=1, expand=True)

# Criar a coluna cpv_division a partir dos dois primeiros números do cpv_código
combined_df['cpv_division'] = combined_df['cpv_código'].str[:2]

# Remover a coluna original
combined_df.drop(columns=['cpv'], inplace=True)

# Função para processar a geografia
def process_geografia(entry):
    if not isinstance(entry, str):
        return 'Desconhecido', 'Desconhecido'
    # Separar múltiplos locais pelo separador "|"
    locais = entry.split(' | ')
    # Extrair apenas NUTS I e NUTS II
    nuts_ii = set()
    for local in locais:
        niveis = local.split(', ')
        if len(niveis) >= 2:
            nuts_ii.add(niveis[1])  # Pegamos no NUTS II
        else:
            nuts_ii.add('Portugal')  # Apenas Portugal (NUTS I)
    
    # Decidir o âmbito_geo
    if len(nuts_ii) == 1:
        if 'Portugal' in nuts_ii:
            return 'Portugal', 'Nacional'
        else:
            return list(nuts_ii)[0], list(nuts_ii)[0]
    else:
        return 'Múltiplos', 'Múltiplos'

# Adicionar barra de progresso com tqdm
tqdm.pandas(desc="Processando geografia")

# Aplicar a função à coluna geografia com tqdm
combined_df[['NUTS', 'ambito_geo']] = combined_df['localExecucao'].progress_apply(process_geografia).apply(pd.Series)

# Resultado final
print(combined_df.head())


Processando geografia:  14%|█▍        | 253748/1751006 [00:00<00:01, 934682.03it/s]


AttributeError: 'float' object has no attribute 'split'

In [3]:
# Save as pickle file
pkl_output_path = "F:/portal_base/data/processed/portal_base_trimmed.pkl"
combined_df.to_pickle(pkl_output_path)